In [55]:
%%bigquery
CREATE SCHEMA IF NOT EXISTS `qwiklabs-gcp-02-3205ce60aeba.weather`;

DROP TABLE IF EXISTS `qwiklabs-gcp-02-3205ce60aeba.weather.weather_data`;
CREATE OR REPLACE EXTERNAL TABLE `qwiklabs-gcp-02-3205ce60aeba.weather.weather_data`
OPTIONS (
  format = 'CSV',
  uris = ['gs://labs.roitraining.com/data-to-ai-workshop/weather_data.csv'],
  skip_leading_rows = 1
);


Query is running:   0%|          |

""


In [56]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [57]:
%%bigquery
CREATE OR REPLACE MODEL `qwiklabs-gcp-02-3205ce60aeba.weather.gemini_model`
REMOTE WITH CONNECTION DEFAULT OPTIONS (
  ENDPOINT = 'gemini-2.0-flash'
);


Query is running:   0%|          |

""


In [58]:
%%bigquery
SELECT TO_JSON_STRING(ml_generate_text_result['candidates'][0]['content']["parts"][0]["text"]) AS generated_text, *
FROM

  ML.GENERATE_TEXT(
    MODEL `qwiklabs-gcp-02-3205ce60aeba.weather.gemini_model`,
    (
      SELECT
        CONCAT(
          'Generate a weather report or warning for ',
          city, ', ', state, ' on ', FORMAT_DATE('%B %d, %Y', date),
          '. Conditions: ', weather_condition,
          '. Temperature: ', CAST(temperature_f AS STRING), '°F. Wind Speed: ', CAST(wind_speed_mph AS STRING), ' mph. ',
          'Precipitation: ', CAST(precipitation_in AS STRING), ' in. Pressure: ', CAST(barometric_pressure_inHg AS STRING), ' inHg. ',
          'Humidity: ', CAST(humidity_percent AS STRING), '%.'
        ) AS prompt
      FROM
        `qwiklabs-gcp-02-3205ce60aeba.weather.weather_data`
    ),
    STRUCT(
      0.7 AS temperature,
      256 AS max_output_tokens,
      TRUE AS flatten_json_output
    )
  );

Executing query with job ID: a62f3044-6b5b-4b5a-a870-a435bd91dee3
Query executing: 0.41s


ERROR:
 400 Subscript access using [STRING] is not supported on values of type STRING at [1:23]; reason: invalidQuery, location: query, message: Subscript access using [STRING] is not supported on values of type STRING at [1:23]

Location: US
Job ID: a62f3044-6b5b-4b5a-a870-a435bd91dee3

